In [ ]:
#Attempting replace FebHD to get higher accuracy without breaking things

In [175]:
from keras.datasets import fashion_mnist
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from Module import *
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import networkx as nx

In [176]:
def get_graph_from_adjacency_matrix(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    return gr

In [179]:
def combine_groupings(comparison_grouping, category):
    new_indices=[]
    for i in range(0,len(list(category))):
        new_indices+=comparison_grouping[list(category)[i]]
    return new_indices

In [180]:
def uniformity_report(index_list):
    index_list=list(set(index_list))
    realvals=torch.zeros(len(index_list))
    for i, index in enumerate(index_list):
        realvals[i]=y_train[index]
    mode=realvals.mode()
    return mode.values, len(y_train[index_list][y_train[index_list]==mode.values])/len(index_list),len(index_list)

In [5]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train=torch.from_numpy(x_train).reshape((-1,784)).float()
x_test=torch.from_numpy(x_test).reshape((-1,784)).float()
y_train=torch.from_numpy(y_train).long()
y_test=torch.from_numpy(y_test).long()
scaler = preprocessing.Normalizer().fit(x_train)
x_train = torch.tensor(scaler.transform(x_train))
x_test= torch.tensor(scaler.transform(x_test))

In [31]:
model1=HDCModel(10,len(x_train[0]),1000,classh=True)

In [32]:
x_train_encoded=model1.encode(x_train)

In [138]:
numerator=0
for i in range(0,100): #repetitions
    rand_index=np.random.randint(0,len(y_train))
    sims=cos_cdist(x_train_encoded,x_train_encoded[rand_index:rand_index+1]).squeeze()
    genus=torch.argsort(-sims)
    categories=[]
    for i in range(0,60):# members in genus
        categories.append(y_train[genus[i]])
    categories=torch.tensor(categories)
    proportion=len(categories[categories == categories.mode().values])/len(categories)
    numerator+=proportion
print(numerator/100)

0.8288333333333333


In [336]:
num_groups=1000

In [337]:
mask=torch.zeros(len(y_train)).bool()
rand_indices=torch.zeros(num_groups).int()
for i in range(num_groups):
    rand_indices[i]=np.random.randint(0,len(y_train))
for i in rand_indices:
    mask[i]=True

In [338]:
sims=cos_cdist(x_train_encoded,x_train_encoded[mask])

In [347]:
groups=[]
for i in range(0,num_groups):
    genus=torch.argsort(-sims[i]).tolist()
    groups.append(genus[:100])

In [348]:
similarity_matrix=torch.zeros(num_groups,num_groups)

In [349]:
for i,list_ in enumerate(groups):
    for j,comparison_list in enumerate(groups):
        similarity_matrix[i][j]=len(set(list_).intersection(set(comparison_list)))

In [350]:
similarity_matrix

tensor([[100.,   0.,   0.,  ...,   0.,  82.,  24.],
        [  0., 100.,  13.,  ...,  16.,   0.,   0.],
        [  0.,  13., 100.,  ...,  44.,   0.,   0.],
        ...,
        [  0.,  16.,  44.,  ..., 100.,   0.,   0.],
        [ 82.,   0.,   0.,  ...,   0., 100.,  17.],
        [ 24.,   0.,   0.,  ...,   0.,  17., 100.]])

In [362]:
adjacency_matrix=similarity_matrix>=100

In [363]:
similarity_matrix

tensor([[100.,   0.,   0.,  ...,   0.,  82.,  24.],
        [  0., 100.,  13.,  ...,  16.,   0.,   0.],
        [  0.,  13., 100.,  ...,  44.,   0.,   0.],
        ...,
        [  0.,  16.,  44.,  ..., 100.,   0.,   0.],
        [ 82.,   0.,   0.,  ...,   0., 100.,  17.],
        [ 24.,   0.,   0.,  ...,   0.,  17., 100.]])

In [364]:
gr=get_graph_from_adjacency_matrix(adjacency_matrix)
connected_components=nx.connected_components(gr)
subgraphs=[gr.subgraph(c) for c in connected_components]

In [365]:
categories=nx.community.louvain_communities(gr, threshold=1e-0, seed=None)
print(len(categories))
for i in range(0,len(categories)):
    print(len(categories[i]))
sum_=0
denom=0
clusters=[]
print(len(categories))
print(categories[0])
for i in range(0,len(categories)):
    print(i)
    cluster=combine_groupings(groups,categories[i])
    denom+=len(cluster)*uniformity_report(cluster)[1]
    sum_+=len(cluster)
    print(uniformity_report(cluster))
    print(len(cluster))
    clusters.append(cluster)
print(denom/sum_)

1000
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

In [58]:
rand_index=np.random.randint(0,len(y_train))

In [59]:
sims=cos_cdist(x_train_encoded,x_train_encoded[rand_index:rand_index+1]).squeeze()
genus=torch.argsort(-sims)

In [61]:
for i in range(0,10):
    print(y_train[genus[i]])

tensor(0)
tensor(0)
tensor(6)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(6)
tensor(0)
tensor(6)
